<a href="https://colab.research.google.com/github/udothemath/ncku_customer_embedding/blob/main/WenMing/0_sample_file_by_chid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is used for data preprocessing and sampling

In [3]:
from google.colab import drive
drive.mount('/gdrive')
!ln -s /gdrive/MyDrive/colab/NCKU_embedding/data/ ./data


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from time import time
import pandas as pd
import numpy as np
import os
import random
from tqdm.notebook import tqdm
import datetime

In [2]:
data_path = './data'
cdtx_path = 'raw/zip_if_cca_cdtx0001_hist'
y_path = 'raw/zip_if_cca_y'
cust_f_path = 'raw/zip_if_cca_cust_f'
stonc_tag_f_path = 'raw/zip_if_cca_stonc_tag_features'

child_file = 'sample_chid.txt'
cdtx_file = 'pickle_if_cca_cdtx0001_hist_{:04d}.pkl'
y_file = 'pickle_if_cca_y_{:04d}.pkl'
cust_f_file = 'pickle_if_cca_cust_f_{:04d}.pkl'
stonc_tag_f_file = 'pickle_if_cca_stonc_tag_features_0000.pkl'

## 進行資料篩選
* 隨機抽取了三個檔案 pickle_if_cca_cdtx0001_hist_0008～10.pkl
* 篩選日期為小於 2019/1/1 

In [5]:
!wc -l data/raw/zip_if_cca_cdtx0001_hist/pickle_if_cca_cdtx0001_hist_0010.pkl
!wc -l data/raw/zip_if_cca_cdtx0001_hist/pickle_if_cca_cdtx0001_hist_0009.pkl
!wc -l data/raw/zip_if_cca_cdtx0001_hist/pickle_if_cca_cdtx0001_hist_0008.pkl

575117 data/raw/zip_if_cca_cdtx0001_hist/pickle_if_cca_cdtx0001_hist_0010.pkl
677589 data/raw/zip_if_cca_cdtx0001_hist/pickle_if_cca_cdtx0001_hist_0009.pkl
10008444 data/raw/zip_if_cca_cdtx0001_hist/pickle_if_cca_cdtx0001_hist_0008.pkl


In [3]:
data1 = np.load(f'data/raw/zip_if_cca_cdtx0001_hist/pickle_if_cca_cdtx0001_hist_0008.pkl',  allow_pickle=True)
data1 = data1[data1.csmdt < datetime.date(2019, 1, 1)]
chids = set(data1.chid.unique())
del data1

In [4]:
data2 = np.load(f'data/raw/zip_if_cca_cdtx0001_hist/pickle_if_cca_cdtx0001_hist_0009.pkl',  allow_pickle=True)
data2 = data2[data2.csmdt < datetime.date(2019, 1, 1)]
chids.update(data2.chid.unique())
del data2

In [5]:
data3 = np.load(f'data/raw/zip_if_cca_cdtx0001_hist/pickle_if_cca_cdtx0001_hist_0010.pkl',  allow_pickle=True)
data3 = data3[data3.csmdt < datetime.date(2019, 1, 1)]
chids.update(data3.chid.unique())
del data3

## 產生所有不重複的 chid (customer ID)
* 並產出 customer id list File : './data/sample_50k/sample_chid.txt'

In [6]:
chids = np.array(list(chids))

In [8]:
random.seed(1012+4028+4036)
sample_chids = random.sample(list(chids), k = 50000)
with open(os.path.join(data_path, 'sample_50k', child_file), 'w') as f:
    for i in sample_chids:
        f.write(i+'\n')

## Produce the mapping table that serial number -> chid

In [9]:
idx_map = {}
for i,j in enumerate(sample_chids):
    idx_map[j] = i

In [10]:
np.save(os.path.join(data_path, 'sample_50k/sample_idx_map'), idx_map)

## Load White List

In [11]:
chid_arary = np.loadtxt(os.path.join(data_path, 'sample_50k', child_file), dtype=np.str)
len(set(chid_arary))

50000

## Counting the total loading time and extrac transation log by chid list 
* this is used for counting the toal time that load whole zip_if_cca_cdtx0001_hist data 
* suffule all pickle file and filtering by chid list

In [ ]:
cdtx_list = []

t = time()
for i in range(0, 30):
    print('In',  cdtx_file.format(i))
    
    t0 = time()
    
    df_cdtx = pd.read_pickle(os.path.join(data_path, cdtx_path, cdtx_file.format(i)))
    t1 = time()
    
    print('\t Load file cost', t1 - t0)
    
    mask = df_cdtx.chid.isin(chid_arary)
    t2 = time()
    
    print('\t Find mask cost', t2 - t1)
    
    cdtx_list.append(df_cdtx.values[:][mask])
    t3 = time()
    
    print('\t One file total cost', t3 - t0, '\n')
    
columns = df_cdtx.columns
print('Whole files total cost', t3 - t)

In pickle_if_cca_cdtx0001_hist_0000.pkl
	 Load file cost 23.543294429779053
	 Find mask cost 1.779041051864624
	 One file total cost 25.504454612731934 

In pickle_if_cca_cdtx0001_hist_0001.pkl
	 Load file cost 27.989659547805786
	 Find mask cost 1.7145671844482422
	 One file total cost 29.893545389175415 

In pickle_if_cca_cdtx0001_hist_0002.pkl
	 Load file cost 33.182138204574585
	 Find mask cost 1.706920862197876
	 One file total cost 35.07976007461548 

In pickle_if_cca_cdtx0001_hist_0003.pkl
	 Load file cost 31.36946725845337
	 Find mask cost 1.834467887878418
	 One file total cost 33.399545192718506 

In pickle_if_cca_cdtx0001_hist_0004.pkl
	 Load file cost 32.769492864608765
	 Find mask cost 1.763176441192627
	 One file total cost 34.719396114349365 

In pickle_if_cca_cdtx0001_hist_0005.pkl
	 Load file cost 25.572811126708984
	 Find mask cost 1.8826203346252441
	 One file total cost 27.68309760093689 

In pickle_if_cca_cdtx0001_hist_0006.pkl
	 Load file cost 27.695298671722412
	

In [ ]:
sample_df_cdtx = pd.DataFrame(np.concatenate(cdtx_list, axis=0), columns=df_cdtx.columns)
sample_df_cdtx = sample_df_cdtx.sort_values(by=['csmdt', 'chid', 'mcc']).reset_index(drop=True)
sample_df_cdtx.shape

(6608174, 10)

In [ ]:
len(sample_df_cdtx.chid.unique()), len(sample_df_cdtx.csmdt.unique()), len(sample_df_cdtx.mcc.unique())

(50000, 761, 518)

In [ ]:
sample_df_cdtx.to_csv(os.path.join(data_path, 'sample_50k', 'sample_zip_if_cca_cdtx0001_hist.csv'), index=False, encoding='utf-8')

In [ ]:
!head -n 3 ./data/sample_50k/sample_zip_if_cca_cdtx0001_hist.csv

bnsfg,bnspt,chid,csmdt,iterm,mcc,objam,scity,tcode,hcefg
N,0,+10kXREN0NI0BQgqeDnT4g==,2018-01-01,0,5411,915,TAICHUNG,05,
N,0,+1cvhSoBR5xt9eHv3bVU8g==,2018-01-01,0,MRT5,2600,,05,


## 對學習目標資料檔進行整理 zip_if_cca_y

In [ ]:
y_list = []

t = time()
for i in range(0, 13):
    print('In',  y_file.format(i))
    
    t0 = time()
    
    df_y = pd.read_pickle(os.path.join(data_path, y_path, y_file.format(i)))
    t1 = time()
    
    print('\t Load file cost', t1 - t0)
    
    mask = df_y.chid.isin(chid_arary)
    t2 = time()
    
    print('\t Find mask cost', t2 - t1)
    
    y_list.append(df_y.values[:][mask])
    t3 = time()
    
    print('\t One file total cost', t3 - t0, '\n')
    
columns = df_y.columns
print('Whole files total cost', t3 - t)

In pickle_if_cca_y_0000.pkl
	 Load file cost 11.947602987289429
	 Find mask cost 1.3693790435791016
	 One file total cost 13.395427465438843 

In pickle_if_cca_y_0001.pkl
	 Load file cost 11.451340675354004
	 Find mask cost 1.4168272018432617
	 One file total cost 12.943442106246948 

In pickle_if_cca_y_0002.pkl
	 Load file cost 10.332145690917969
	 Find mask cost 1.3855071067810059
	 One file total cost 11.813363552093506 

In pickle_if_cca_y_0003.pkl
	 Load file cost 9.59541130065918
	 Find mask cost 1.438009262084961
	 One file total cost 11.129197597503662 

In pickle_if_cca_y_0004.pkl
	 Load file cost 9.628886222839355
	 Find mask cost 1.4116389751434326
	 One file total cost 11.135394811630249 

In pickle_if_cca_y_0005.pkl
	 Load file cost 9.532246112823486
	 Find mask cost 1.3984348773956299
	 One file total cost 11.024503231048584 

In pickle_if_cca_y_0006.pkl
	 Load file cost 9.630181312561035
	 Find mask cost 1.4090349674224854
	 One file total cost 11.133630275726318 

In pi

In [ ]:
sample_df_y = pd.DataFrame(np.concatenate(y_list, axis=0), columns=df_y.columns)
sample_df_y = sample_df_y.sort_values(by=['data_dt', 'chid', 'stonc_tag']).reset_index(drop=True)
sample_df_y.shape

(2877419, 4)

In [ ]:
len(sample_df_y.chid.unique()), len(sample_df_y.data_dt.unique()), len(sample_df_y.stonc_tag.unique())

(50000, 24, 49)

In [ ]:
sample_df_y.to_csv(os.path.join(data_path, 'sample_50k', 'sample_zip_if_cca_y.csv'), index=False, encoding='utf-8')

### the objective data include below column
* date_dt: the transaction's date
* chid: user's id
* stonc_tg: merchant category
* y: the amount of consumption



In [ ]:
!head -n 5 ./data/sample_50k/sample_zip_if_cca_y.csv

data_dt,chid,stonc_tag,y
2018-01-01,++09R1U4V8zedf5TUopHeQ==,D1,14737
2018-01-01,++4jH9yOMLHOVNVM4oP8IA==,R2,27274
2018-01-01,++4jH9yOMLHOVNVM4oP8IA==,TR1,2137
2018-01-01,++8SyhBUgS2w3Iks4+Galg==,I1,10296


## 根據抽樣過後的 chid list 取出相對的使用者 Profile

In [ ]:
pd.read_pickle(os.path.join(data_path, cust_f_path, cust_f_file.format(0))).head(3)

,chid,data_ym,monin,wrky,first_mob,data_dt,masts,educd,naty,trdtp,...,constant_u2_ind,constant_u3_ind,constant_u4_ind,constant_l2_ind,constant_l3_ind,constant_l4_ind,constant_change,growth_rate,monotone_up,monotone_down
0,O+zDrRhv99Natp0tP+0xnA==,2017-12-01,163270,0,191.0,2018-01-01,ea9d465d7361343a138603660b263e4f9fdb7bc04c5843...,1ed13fb8d916b8ba4aca01f3ab6eca7a819b92e912c7a8...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,f0243f3eded4a032b3ad01b034d5d6a96b92d8ba6f7344...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,dHPu/DMPPsGh3cVQSdeYPQ==,2017-12-01,0,0,187.0,2018-01-01,ea9d465d7361343a138603660b263e4f9fdb7bc04c5843...,9ea8636e8661c94a50a8f80087137df3df35659c2aca17...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,f0243f3eded4a032b3ad01b034d5d6a96b92d8ba6f7344...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,R7Qu5/8UhOxSfMTgDRqYYw==,2017-12-01,0,0,200.0,2018-01-01,ea9d465d7361343a138603660b263e4f9fdb7bc04c5843...,1ed13fb8d916b8ba4aca01f3ab6eca7a819b92e912c7a8...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,99bc9b817c9067f33b85455a28efaba660e6aa1231be75...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
cust_f_list = []

t = time()
for i in range(0, 94):
    print('In',  cust_f_file.format(i))
    
    t0 = time()
    
    df_cust_f = pd.read_pickle(os.path.join(data_path, cust_f_path, cust_f_file.format(i)))
    t1 = time()
    
    print('\t Load file cost', t1 - t0)
    
    mask = df_cust_f.chid.isin(chid_arary)
    t2 = time()
    
    print('\t Find mask cost', t2 - t1)
    
    cust_f_list.append(df_cust_f.values[:][mask])
    t3 = time()
    
    print('\t One file total cost', t3 - t0, '\n')
    
columns = df_cust_f.columns
print('Whole files total cost', t3 - t)

In pickle_if_cca_cust_f_0000.pkl
	 Load file cost 13.440500974655151
	 Find mask cost 0.19030404090881348
	 One file total cost 13.682416677474976 

In pickle_if_cca_cust_f_0001.pkl
	 Load file cost 22.394106149673462
	 Find mask cost 0.18876028060913086
	 One file total cost 22.637964963912964 

In pickle_if_cca_cust_f_0002.pkl
	 Load file cost 19.273138999938965
	 Find mask cost 0.20464491844177246
	 One file total cost 19.52684998512268 

In pickle_if_cca_cust_f_0003.pkl
	 Load file cost 15.229641675949097
	 Find mask cost 0.20006656646728516
	 One file total cost 15.485936403274536 

In pickle_if_cca_cust_f_0004.pkl
	 Load file cost 17.650744438171387
	 Find mask cost 0.19668030738830566
	 One file total cost 17.91371250152588 

In pickle_if_cca_cust_f_0005.pkl
	 Load file cost 15.982235193252563
	 Find mask cost 0.19440793991088867
	 One file total cost 16.237755060195923 

In pickle_if_cca_cust_f_0006.pkl
	 Load file cost 17.523010730743408
	 Find mask cost 0.19675660133361816
	 

	 Load file cost 15.625621318817139
	 Find mask cost 0.21619868278503418
	 One file total cost 15.910445928573608 

In pickle_if_cca_cust_f_0057.pkl
	 Load file cost 15.364200353622437
	 Find mask cost 0.2230381965637207
	 One file total cost 15.654796123504639 

In pickle_if_cca_cust_f_0058.pkl
	 Load file cost 15.05195665359497
	 Find mask cost 0.21611356735229492
	 One file total cost 15.335163116455078 

In pickle_if_cca_cust_f_0059.pkl
	 Load file cost 14.677219867706299
	 Find mask cost 0.22040152549743652
	 One file total cost 14.956663131713867 

In pickle_if_cca_cust_f_0060.pkl
	 Load file cost 15.044909715652466
	 Find mask cost 0.22010135650634766
	 One file total cost 15.321080684661865 

In pickle_if_cca_cust_f_0061.pkl
	 Load file cost 14.927440166473389
	 Find mask cost 0.21578431129455566
	 One file total cost 15.198678255081177 

In pickle_if_cca_cust_f_0062.pkl
	 Load file cost 14.913427829742432
	 Find mask cost 0.21657466888427734
	 One file total cost 15.1911010742

In [ ]:
cust_f_list

[array([['bcNzZ56EW6Vtp0gA3sRTQQ==', datetime.date(2017, 12, 1),
         Decimal('210610'), ..., Decimal('1.6'), Decimal('1.0'),
         Decimal('0.0')],
        ['jdSXh0NqtTo+2w7feMhbTw==', datetime.date(2017, 12, 1),
         Decimal('120338'), ..., Decimal('5.5'), Decimal('1.0'),
         Decimal('0.0')],
        ['27FXhTWD18Nhuv6XDGxP8w==', datetime.date(2017, 12, 1),
         Decimal('292334'), ..., Decimal('1.2'), Decimal('5.0'),
         Decimal('0.0')],
        ...,
        ['MXQf7/VCYcqQzbLx31DrmQ==', datetime.date(2017, 12, 1),
         Decimal('432208'), ..., Decimal('0.8'), Decimal('0.0'),
         Decimal('1.0')],
        ['uBTEg/o7h06vGrDNTKXdqQ==', datetime.date(2017, 12, 1),
         Decimal('480360'), ..., Decimal('0.9'), Decimal('0.0'),
         Decimal('1.0')],
        ['1DNXdueeCE/V5VVV73EodQ==', datetime.date(2017, 12, 1),
         Decimal('196742'), ..., Decimal('0.8'), Decimal('0.0'),
         Decimal('6.0')]], dtype=object),
 array([['T3WHQeip8TqnXysnlB860g=='

In [ ]:
sample_df_cust_f = pd.DataFrame(np.concatenate(cust_f_list, axis=0), columns=df_cust_f.columns)
sample_df_cust_f = sample_df_cust_f.sort_values(by=['chid', 'data_ym', 'data_dt']).reset_index(drop=True)
sample_df_cust_f.shape

(1188902, 32)

In [ ]:
len(sample_df_cust_f.chid.unique()), len(sample_df_cust_f.data_ym.unique()), len(sample_df_cust_f.data_dt.unique())

(50000, 24, 24)

In [ ]:
sample_df_cust_f.to_csv(os.path.join(data_path, 'sample_50k', 'sample_zip_if_cca_cust_f.csv'), index=False, encoding='utf-8')

In [ ]:
!head -n 3 ./data/sample_50k/sample_zip_if_cca_cust_f.csv

chid,data_ym,monin,wrky,first_mob,data_dt,masts,educd,naty,trdtp,poscd,cuorg,cycam,slam,sum_area_c,sum_u2_ind,sum_u3_ind,sum_u4_ind,sum_l2_ind,sum_l3_ind,sum_l4_ind,constant_area_c,constant_u2_ind,constant_u3_ind,constant_u4_ind,constant_l2_ind,constant_l3_ind,constant_l4_ind,constant_change,growth_rate,monotone_up,monotone_down
++03Yg+R1oaKBt0f2gWCgA==,2017-12-01,211645,0,80.0,2018-01-01,ea9d465d7361343a138603660b263e4f9fdb7bc04c5843934b404e36159b0936,6ae4ba148e6b8652e0768cf3db4bbade8ffc50c94d08a5da3fbc919cb0de744e,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f6624f0d445428a73cdf0,f0243f3eded4a032b3ad01b034d5d6a96b92d8ba6f7344058f15138bab08d1f7,78c2e56448aad92ca71e8501f14fdc6f4a177ee7773dc4df3e7311b7a41ffc99,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019468e3a67e7b6ab2da2,1040000,1352000,2.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.8,0.0,1.0
++03Yg+R1oaKBt0f2gWCgA==,2018-01-01,211645,0,81.0,2018-02-01,ea9d465d7361343a138603660b263e4f9fdb7bc04c5843934b404e36159b0936,6ae4